In [29]:
from tqdm import tqdm
import pandas as pd

# Data pre-processing

## Filter out unwanted rows

In [30]:
# parameters & functions
cols_to_read = ['user_id', 'path', 'normalized_path', 'method']
cols_to_write = ['user_id', 'path', 'normalized_path', 'method']
endpoints_list = [
    '/streams/{streamid}/posts',
    '/streams/{streamid}/subscribe',
    '/streams/{streamid}/members',
    '/streams/{streamid}/subscribe',
    
    '/posts/{postid}/comments',
    '/posts',
    '/posts/{postid}/like',
    '/posts/{postid}/likes',
    '/posts/{postids}/read',
    '/posts/{post_id}/vote/{option_id}',
    
    '/comments/{id}/likes',
    '/comments/{id}/keep',
    '/comments/{id}/like',
]
methods_list = ['POST']

def filter_rows(path, method):
    return (not pd.isnull(path) # path must not be NaN
            and not pd.isnull(method) # method must not be NaN
            and any(endpoint in path for endpoint in endpoints_list) # the path must contain one of the wanted endpoints
            and method in methods_list) # the method should be one of the wanted methods

# read data
data = pd.read_csv('./data/hospitality_dataset_2020.csv', chunksize=60, usecols=cols_to_read) # data is an iterable

header = True
for chunk in tqdm(data):
    # filter out unwanted rows
    chunk = chunk[chunk[['normalized_path', 'method']].apply(lambda x: filter_rows(*x), axis=1)]
    # save remaining rows
    chunk.to_csv('./data/trimmed_hospitality_dataset_2020.csv', header=header, columns=cols_to_write, index=False, mode='a')
    header = False # only first chunk should have a header

830563it [1:41:43, 136.08it/s]
